In [3]:
# import pandana
# import osmnx as ox
import geopandas as gpd
from sqlalchemy import create_engine
import psycopg2
import pandas as pd
import geoalchemy2
import numpy as np

In [5]:
df = gpd.read_file('h3_dubay.geojson')

In [8]:
buildings_1 = gpd.read_file('buildings\Buildings_1_sector.geojson')
buildings_2 = gpd.read_file('buildings\Buildings_2_sector.geojson')
buildings_3 = gpd.read_file('buildings\Buildings_3_sector.geojson')
buildings_4 = gpd.read_file('buildings\Buildings_4_sector.geojson')
buildings_5 = gpd.read_file('buildings\Buildings_5_sector.geojson')
buildings_6 = gpd.read_file('buildings\Buildings_6_sector.geojson')
buildings_7 = gpd.read_file('buildings\Buildings_7_sector.geojson')
buildings_8 = gpd.read_file('buildings\Buildings_8_sector.geojson')
buildings_9 = gpd.read_file('buildings\Buildings_9_sector.geojson')

In [10]:
buildings = pd.concat([buildings_1, buildings_2, buildings_3, buildings_4, buildings_5, buildings_6, buildings_7, buildings_8, buildings_9])

In [11]:
buildings.head(5)

,id,name,amenity,housenumbe,street,building,levels,shop,leisure,country,place,tourism,historic,feature_id,area,code,periods,geometry
0,215463,Al-Yasmeen Building,NaN,NaN,NaN,yes,5.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1970-1980,"MULTIPOLYGON (((333277.850 2796572.843, 333348..."
1,2781584,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,1960-1970,"MULTIPOLYGON (((330399.833 2797167.772, 330435..."
2,2781585,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101,NaN,NaN,1960-1970,"MULTIPOLYGON (((330653.801 2797419.646, 330657..."
3,3336311,Lotus hotel,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,hotel,NaN,122,NaN,NaN,1970-1980,"MULTIPOLYGON (((330691.108 2795309.983, 330661..."
4,3336312,Summit Hotel,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,hotel,NaN,123,NaN,NaN,1970-1980,"MULTIPOLYGON (((330803.434 2795277.380, 330766..."


In [14]:
buildings['geometry'] = buildings['geometry'].to_crs(3857)

In [15]:
buildings_hex = gpd.sjoin(buildings, df, how='left', predicate='intersects', lsuffix='left', rsuffix='right')

In [19]:
counts = buildings_hex['h3'].value_counts()
counts = counts.reset_index().rename(columns={"index": "h3", "h3": "count"})
# buildings_hex['count'] = df['h3'].map(counts)

In [24]:
counts

# df.rename(columns={"A": "a", "B": "b", "C": "c"}, errors="raise")

,h3,count
0,8843accaabfffff,2478
1,8843acca3bfffff,1866
2,8843accabdfffff,1796
3,8843acca51fffff,1487
4,8843accaedfffff,1462
...,...,...
1967,8843ac503dfffff,1
1968,8843ac502bfffff,1
1969,8843ac5065fffff,1
1970,8843acc10dfffff,1


In [32]:
buildings_hex = buildings_hex.merge(counts, on = 'h3', how = 'left').rename(columns={"count_y": "count_building"})

In [34]:
buildings_hex['population_house'] = buildings_hex['population']/ buildings_hex['count_building']

In [36]:
buildings_hex['population_house_round'] = buildings_hex['population_house'].round()

# data['rounded_numbers'] = data['numbers'].round(2)

In [38]:
buildings_hex.to_file('buildings_hex.geojson')